---
title: "2023_Kronensicherung_Plesse_Kraefte_Schwingungen"
author: "Kyell Jensen"
date: "2024-08-06"
format: pdf
editor: visual
---

# 2023_Kronensicherung_Plesse_Kraefte_Schwingungen

## Kombinierte Analyse LineScale3, TreeQinetic und Versuchsaufzeichung

Nutze eine geeignete Python 3.11 Umgebung (z. B. virtuelle Environment) und installiere die Pakete linescale3 (LS3) und treeqinetic (PTQ) inklusive kj_core und kj_logger und weiteren requirements.

## Package Importe

Es werden zuerst benötigte Standard-Pakete importiert. Nachfolgend die zwei extra geschriebenen Pakete LS3 und PTQ. Fehler beim Import dieser zwei Pakete sind ggf. Bugs. Beide Pakete nutzen eine gemeinsame CodeBasis in den Paketen kj_core (Core-Package) und kj_logger (individualisiertes Logging des Verarbeitungs-Prozesses). Diese sollte i. d. R. über die requirements mit installiert werden.

### Import von Standardbibliotheken

Die folgenden Bibliotheken werden importiert, um grundlegende Funktionen für Strukturierung, Datenverarbeitung, Plotting und statistische Auswertung bereit zu stellen.

In [19]:
# Struktur
from pathlib import Path
from typing import Dict, List

# Datenverarbeitung
import json
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
from slugify import slugify  # Slugify ums strings in standard Formate zu überführen

# Plotting
import seaborn as sns
import matplotlib.pyplot as plt

# Statistik
from scipy.stats import linregress, f_oneway
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.multicomp as mc


## Import TreeQinetic

Das Packet TreeQinetic wurde vom Autor (Kyell Jensen) zum einfachen Analysieren, Plotten und zur Interpretation der TXT-Messdaten der Picus TreeQinetic Elastometer und Inclinometer der Firma IML Instrumenta Mechanik Labor Electronic GmbH geschrieben (https://www.iml-electronic.de/produkt/picus-treeqinetic/). Nachfolgend wird das Packet und einige dort definierten Klassen importiert.

In [21]:
import treeqinetic as ptq
# ptq.help() # Test

## Daten Import und explorative Datenanalyse

Lege Pfade für Daten-Importe, Daten-Exporte etc. fest (ggf. anpassen an eigene Verzeichnisstruktur), ausgelagert in gemeinsame Config für verschiedene Notebooks

In [22]:
# Importiere alle Einstellungen aus der project_config.py
from project_config import (
    main_path,
    analyse_name,
    data_path,
    working_directory,
    data_export_directory,
    latex_export_directory
)

### METADATA: Lade Metadaten zum Versuch (Baumdaten, Geräteanordnung, Versuchsablauf)

Die Daten wurden während des Versuchs als GoogleSheet erfasst und abschließend als CSV-Dateien exportiert, um ein einfaches Einlesen zur ermöglichen.
Nachfolgend werden Datentypen für alle Spalten explizit in den *_data_dict.json angegeben, da es teils durch Python-Pandas zu Fehlerkennungen kommt.
Zusätzlich werden teils Spaltennamen vereinheitlicht oder benannt.

#### Daten des Versuchsbaums (tree_df)

Die Tabelle `tree_df` liefert spezifische Informationen über den Versuchsbaum. Da nur ein Versuchsbaum, sind hier Daten und Datendokumentation in einer Datei.

In [23]:
dtype_dict = {
    'Attribut': 'string',
    'Wert': 'string',
    'Einheit': 'string'
}
tree_file = data_path / 'tree.csv'
tree_df = pd.read_csv(tree_file,
                      sep=';', decimal=',', na_values='NA',
                      dtype=dtype_dict)
tree_df

,Variable,Kategorie,Zeichen,Deutsch,Datentyp,Einheit,Beschreibung,Wert
0,baumart,tree,Baumart,Baumart,string,-,Art des Baumes,Fagus silvatica
1,belaubung,tree,Belaubung,Belaubung,string,-,Belaubungszustand zur Messung,unbelaubt
2,vitalitaet,tree,Vitalitaet,Vitalität nach A. Roloff,Int64,Stufe,Vitalitätsstufe nach Andreas Roloff,1
3,h_st_a,tree,$h_{\mathrm{StA}}$,Höhe Stämmling A,float64,m,Gesamthöhe des Stämmlings A,26.15
4,h_st_b,tree,$h_{\mathrm{StB}}$,Höhe Stämmling B,float64,m,Gesamthöhe des Stämmlings B,27.20
5,u_1m,tree,"$u_{1\,\mathrm{m}}$",Umfang auf 1~m,float64,cm,Stammumfang auf 1~m Höhe,140.00
6,h_zwiesel,tree,$h_{\mathrm{Zwiesel}}$,Höhe Zwiesel,float64,m,Höhe des Zwiesels über Boden,10.31
7,h_ks,tree,$h_{\mathrm{KS}}$,Höhe Kronensicherung,float64,m,Höhe der KS-Anbringung über Boden,17.40
8,l_ks,tree,$l_{\mathrm{KS}}$,Länge Kronensicherung,float64,cm,Länge der eingebauten KS,135.00
9,u_st_a_h_ks,tree,"$u_{\mathrm{StA,KS}}$",Umfang Stämmling A auf Höhe KS,float64,cm,Stammumfang von A auf Höhe der KS,58.00


In [24]:
# Alle Spalten pauschal als string lesen
tree_df = pd.read_csv(
    data_path / 'tree.csv',
    sep=';', decimal=',',
    na_values='NA',
    dtype='string'  # alle Spalten als string
)

tree_df

,Variable,Kategorie,Zeichen,Deutsch,Datentyp,Einheit,Beschreibung,Wert
0,baumart,tree,Baumart,Baumart,string,-,Art des Baumes,Fagus silvatica
1,belaubung,tree,Belaubung,Belaubung,string,-,Belaubungszustand zur Messung,unbelaubt
2,vitalitaet,tree,Vitalitaet,Vitalität nach A. Roloff,Int64,Stufe,Vitalitätsstufe nach Andreas Roloff,1
3,h_st_a,tree,$h_{\mathrm{StA}}$,Höhe Stämmling A,float64,m,Gesamthöhe des Stämmlings A,26.15
4,h_st_b,tree,$h_{\mathrm{StB}}$,Höhe Stämmling B,float64,m,Gesamthöhe des Stämmlings B,27.20
5,u_1m,tree,"$u_{1\,\mathrm{m}}$",Umfang auf 1~m,float64,cm,Stammumfang auf 1~m Höhe,140.00
6,h_zwiesel,tree,$h_{\mathrm{Zwiesel}}$,Höhe Zwiesel,float64,m,Höhe des Zwiesels über Boden,10.31
7,h_ks,tree,$h_{\mathrm{KS}}$,Höhe Kronensicherung,float64,m,Höhe der KS-Anbringung über Boden,17.40
8,l_ks,tree,$l_{\mathrm{KS}}$,Länge Kronensicherung,float64,cm,Länge der eingebauten KS,135.00
9,u_st_a_h_ks,tree,"$u_{\mathrm{StA,KS}}$",Umfang Stämmling A auf Höhe KS,float64,cm,Stammumfang von A auf Höhe der KS,58.00


#### Daten der Geräteanordnung am Baum (sensor_df)

Die `sensor_position.csv` enthält detaillierte Informationen zur Anordnung der Sensoren an den Bäumen im Rahmen des Experiments. Jede Zeile beschreibt die Platzierung eines Sensors. Die Datendokumentation ergibt sich aus `sensor_position_data_dict.json`.

In [25]:
# Lade das Dictionary mit der Daten Dokumentation
with open(data_path/ "sensor_position_data_dict.json", "r", encoding="utf-8") as f:
    sensor_position_dict = json.load(f)

# Erzeuge dtype_dict dynamisch aus dem der Daten Dokumentation
dtype_dict = {key: value["Datentyp"] for key, value in sensor_position_dict.items()
    if value["Datentyp"] not in [None, ""]
}

# CSV einlesen mit dynamischen Datentypen
sensor_position_file = data_path / 'sensor_position.csv'
sensor_df = pd.read_csv(sensor_position_file, sep=';', decimal=',', na_values='NA',dtype=dtype_dict)

sensor_df["height"] = sensor_df["height"] / 100
sensor_df["circumference"] = sensor_df["circumference"] / 100
sensor_df["diameter"] = sensor_df["circumference"] / np.pi

sensor_df.head()

,position_id,type,sensor_id,location,height,circumference,direction,note,diameter
0,1,LS3,14:BF:E6,rope,18.3,<NA>,<NA>,<NA>,<NA>
1,2,LS3,14:99:1E,cable,17.4,<NA>,<NA>,<NA>,<NA>
2,3,TMS1,015,StA,18.0,0.45,west,<NA>,0.143239
3,4,TMS1,014,StB,18.0,0.4,west,<NA>,0.127324
4,5,TMS1,013,StA,15.0,0.67,west,<NA>,0.213268


In [26]:
#sensor_df.dtypes

#### Daten des Versuchsablaufs (series_df)

Die `series.csv` enthält das Protokoll des Versuchsablaufes der einzelnen Messungen. Jede Zeile repräsentiert eine einzelne Messung. Die Datendokumentation ergibt sich aus `series_data_dict.py`.

##### Behandlungsvariante/Kronensicherung Kategorien

- **`free`**: Der Baum konnte frei ohne Kronensicherung nach dem Release ausschwingen.
  
- **`gefa_dynamic`**: In ca. 2/3 der Baumhöhe wurde ein dynamisches Gefa Gurtband 4t dynamisch nach ZTV-Baumpflege mit leichtem Durchhang installiert. Das Ausschwingen wurde durch die KS abgedämpft. Da die Sicherung ohne Vorspannung installiert wurde, zeigen die Plots (/ls3/plots/force_vs_time_1/) von '14:99:1E' sowohl am Anfang als auch am Ende ca. 0 kN Kraft an (LogNr 1 bis 9).

- **`cobra_static`**: In ca. 2/3 der Baumhöhe wurde eine statische Cobra ultrastatic 7t (Dyneema) Sicherung installiert. Die Vorspannung betrug ca. 0,4 kN, wie sich in den Plots (/ls3/plots/force_vs_time_1/) von '14:99:1E' gut erkennen lässt. Durch das Zusammenziehen der Stämmlinge ist die Kronensicherung vor dem Release vollständig lastfrei (0 kN). Nach dem Release pendelt sich die Kraft ca. bei 0,4 kN ein (LogNr 10-18).

- **`cobra_static_slack`**: Ähnlich der `cobra_static`, jedoch wurde die Vorspannung entfernt. Aufgrund von Regen wurde nur eine Messung durchgeführt und die Serie frühzeitig abgebrochen.


In [27]:
# Lade das Dictionary mit der Daten Dokumentation
with open(data_path / "series_data_dict.json", "r", encoding="utf-8") as f:
    series_dict = json.load(f)

# Erzeuge dtype_dict dynamisch aus der Daten Dokumentation
dtype_dict = {key: value["Datentyp"] for key, value in series_dict.items()
              if value["Datentyp"] not in [None, ""]
}

# CSV einlesen mit dynamischen Datentypen
series_file = data_path / 'series.csv'
series_df = pd.read_csv(series_file, sep=';', decimal=',', na_values='NA', dtype=dtype_dict)

# Zeitspalte in Uhrzeit umwandeln
series_df["time"] = pd.to_datetime(series_df["time"], format="%H:%M:%S").dt.time

# Behandlungskategorien sortiert definieren
treatment_order = ['free', 'gefa_dynamic', 'cobra_static', 'cobra_static_slack']
treatment_category = CategoricalDtype(categories=treatment_order, ordered=True)
series_df["treatment"] = series_df["treatment"].astype(treatment_category)

# Vorschau
series_df.head()

,id,time,release_force_target,treatment,14:BF:E6,14:99:1E
0,1,10:10:00,2.5,free,1,<NA>
1,2,10:20:00,2.8,free,2,<NA>
2,3,10:28:00,2.8,free,3,<NA>
3,4,10:47:00,2.8,free,4,<NA>
4,5,10:53:00,2.4,free,5,<NA>


In [28]:
#series_df.dtypes

### Latex Export: Daten und Datendokumentation als Latex-Tabellen

Definiere eine allgemeine Export-Funktion, um die Tabellen als eigene Dateien zu speichern und in Latex zu importieren

In [29]:
from utils_latex_export import (
    save_latex_table,
    extract_latex_dict_from_json
)

#### Latex Export: Daten des Versuchsbaums (tree_df)

Daten und Datendokumentation in einem:

In [30]:
select_data_fields = ["Zeichen", "Deutsch", "Wert", "Datentyp", "Einheit"]
tree_df_latex = tree_df[select_data_fields].copy()

latex_string = tree_df_latex.to_latex(index=False, escape=False, column_format="llrrr", 
                                      float_format="{:0.2f}".format)
caption = "Feldversuch 2 - Daten des Versuchsbaums"

save_latex_table(latex_string, caption, latex_export_directory)

Content saved to: C:\kyellsen\005_Projekte\2024_BA\032_Feldversuch_2023_Plesse\030_Analysen\2023_Kronensicherung_Plesse_Kraefte_Schwingungen\working_directory\export_latex\feldversuch_2_daten_des_versuchsbaums.tex


#### Latex Export: Daten der Geräteanordnung am Baum (sensor_df)

In [31]:
select_data_fields = ["Variable", "Zeichen", "Deutsch", "Datentyp", "Einheit", "Beschreibung"]
sensor_select_variables = ["type", "sensor_id", "location", "direction", "height", "circumference", "diameter"]

Daten:

In [32]:
sensor_df_latex = sensor_df.copy()[sensor_select_variables]

# Ersetze Spaltennamen durch Formelzeichen
sensor_df_latex.columns = [sensor_position_dict[col]["Zeichen"] for col in sensor_select_variables]

latex_string = sensor_df_latex.to_latex(index=False, escape=False, column_format="lllrrrr",
    float_format="{:0.2f}".format)

caption = "Feldversuch 2 - Geräteanordnung"
save_latex_table(latex_string, caption, latex_export_directory)

Content saved to: C:\kyellsen\005_Projekte\2024_BA\032_Feldversuch_2023_Plesse\030_Analysen\2023_Kronensicherung_Plesse_Kraefte_Schwingungen\working_directory\export_latex\feldversuch_2_gerateanordnung.tex


Datendokumentation:

In [33]:
sensor_data_dict_df = extract_latex_dict_from_json(sensor_position_dict, sensor_select_variables, select_data_fields)
latex_string = sensor_data_dict_df.to_latex(index=False, escape=False)
caption = "Feldversuch 2 - Geräteanordnung Daten Dokumentation"

save_latex_table(latex_string, caption, latex_export_directory)

Content saved to: C:\kyellsen\005_Projekte\2024_BA\032_Feldversuch_2023_Plesse\030_Analysen\2023_Kronensicherung_Plesse_Kraefte_Schwingungen\working_directory\export_latex\feldversuch_2_gerateanordnung_daten_dokumentation.tex


#### Latex Export: Daten des Versuchsablaufs (series_df)

In [34]:
series_select_variables = ["id", "time", "release_force_target", "treatment", "14:BF:E6", "14:99:1E"]

Daten:

In [35]:
series_df_latex = series_df.copy()[series_select_variables]

# Ersetze Spaltennamen durch Formelzeichen
series_df_latex.columns = [series_dict[col]["Zeichen"] for col in series_select_variables]

series_df_latex["treatment"] = series_df_latex["treatment"].apply(
    lambda x: "\\texttt{" + str(x).replace("_", "\\_") + "}"
)

latex_string = series_df_latex.to_latex(index=False, escape=False, column_format="llrlrr",
                                        float_format="{:0.2f}".format)

caption = "Feldversuch 2 - Versuchsablauf"
caption_long = "Feldversuch 2 - Versuchsablauf"

save_latex_table(latex_string, caption, latex_export_directory, caption_long=caption_long)

Content saved to: C:\kyellsen\005_Projekte\2024_BA\032_Feldversuch_2023_Plesse\030_Analysen\2023_Kronensicherung_Plesse_Kraefte_Schwingungen\working_directory\export_latex\feldversuch_2_versuchsablauf.tex


Datendokumentation:

In [36]:
series_data_dict_df = extract_latex_dict_from_json(series_dict, series_select_variables, select_data_fields)

latex_string = series_data_dict_df.to_latex(index=False, escape=False)
caption = "Feldversuch 2 - Versuchsablauf Daten Dokumentation"

save_latex_table(latex_string, caption, latex_export_directory)

Content saved to: C:\kyellsen\005_Projekte\2024_BA\032_Feldversuch_2023_Plesse\030_Analysen\2023_Kronensicherung_Plesse_Kraefte_Schwingungen\working_directory\export_latex\feldversuch_2_versuchsablauf_daten_dokumentation.tex


## PTQ: Daten der Elastometer (PicusTreeQinetic Data = PTQ)

### PTQ: Daten Import

Aus dem Zugversuchsset wurden 4 Elastometer und 3 Inclinometer verwendet.

Die Elastometer waren auf dem rechten und linken Stämmling auf der Außenseite in zwei Ebenen platziert. Beim Zusammenziehen der Stämmlinge messen diese entsprechnd eine Faserdehnung, beim Ausschwingen der Stämmlinge über ihre Ruhelage hinaus nach außen eine Faserstauchung.

Die Inclinometer Daten werden hier ebefalls geladen, erschienen aber weniger geeignet zur Auswertung und werden entspricht nicht weiter berücksichtigt.

Die Funktion 'ptq.setup' erstellt div. Instanzen, die für das Paket notwendig sind (CONFIG, LOG_MANAGER, PLOT_MANAGER). 

Über die Klasse 'ptq.Series' wird eine neue Messreihe initialisiert und als 'ptq_series' gespeichert. Im Verzeichnis ptq_data_path finden sich die PTQ Daten als TXT von insgesamt 29 Messungen. Eine Datei enthält jeweils die Daten für alle Inclinometer und Elastometer.

In [ ]:
ptq.setup(working_directory=working_directory, log_level="info", safe_logs_to_file=True)

ptq_data_path = data_path / 'PTQ/data_txt'
ptq_series = ptq.classes.Series(name=analyse_name, path=ptq_data_path)

elasto_names = ["Elasto(95)", "Elasto(98)", "Elasto(92)", "Elasto(90)"]

### PTQ: Explorative Datenanalyse

Übersicht über alle vom PTQ erfassten Daten über alle Messungen gemeinsam (Elastos und Inclinos).

In [ ]:
ptq_df = ptq_series.get_measurements_df()
ptq_df.describe()

#### PTQ: Einhaltung der maximalen Faserdehnung

In [ ]:
ptq_df_elasto_summary = ptq_df[elasto_names].describe()
ptq_df_elasto_summary

Analyse maximaler Dehnung und Stauchung, Prüfung auf Einhaltung der Elastizitätsgrenze (Fagus s. ca. 520 µm über 200~mm Länge des Elastos). 
Elasto(90) mit 429.6 mit der maximalen Faserdehnung

In [ ]:
ptq_df_elasto_min_max = ptq_df_elasto_summary.loc[['min', 'max']]
ptq_df_elasto_min_max

In [ ]:
latex_string = ptq_df_elasto_min_max.to_latex(index=True, escape=True, column_format="r|rrrr", float_format="{:0.2f}".format)

caption = "Feldversuch 2 - Ergebnisse, Maximale Dehnungswerte der Elastometer"
caption_long = "Feldversuch 2 - Ergebnisse, Maximale Dehnungswerte der Elastometer in µm über die Länge der Messgeräte von 200~mm, jeweils der maximale gemessene Werte über alle Messungen, positive Werte = Faserdehnung, negative Werte = Faserstauchung"
save_latex_table(latex_string, caption, latex_export_directory, caption_long)

Für jedes Elasto innerhalb jeder Messung getrennt. Werte werden bei der Gesamtauswertung später verwendet.

In [ ]:
# Gruppiere nach ID und berechne min und max für jede Elasto-Spalte
ptq_df_elasto_min_max = ptq_df.groupby('ID')[elasto_names].agg(['min', 'max'])
ptq_df_elasto_min_max.head(3)

In [ ]:
latex_string = ptq_df_elasto_min_max.reset_index().to_latex(index=False, escape=True, column_format="r|rr|rr|rr|rr", float_format="{:0.2f}".format)

caption = "Feldversuch 2 - Ergebnisse, Maximalwerte Elastometer vollständig"
caption_long = "Feldversuch 2 - Ergebnisse, Maximalwerte Elastometer vollständig, Faserstauchung als min., Faserdehnung als max. in µm über 200~mm Länge Messgerät"
save_latex_table(latex_string, caption, latex_export_directory, caption_long)

Die maximalen Dehnungs- und Stauchungswerte der Elastometer werden in der Struktur angepasst, um später mit den anderen Daten zusammenzuführen.

In [ ]:
ptq_df_elasto = ptq_df_elasto_min_max.stack(level=0, future_stack=True).reset_index()

ptq_df_elasto = ptq_df_elasto.rename(columns={
    'ID': 'id', 
    'level_1': 'sensor_name',
    'max': 'max_strain',
    'min': 'max_compression'
})

ptq_df_elasto.head(10)

#### PTQ: Plotten aller Messungen bzw. Elastometer

Plotten der 4 verwendeten Elastometer in einem Plot für jede Messung. Die Plots werden im Verzeichnis ptq/plots/multi_sensors_vs_time_1/ abgelegt.
Für alle Messungen und Elastometer ist gut zu erkennen, wie die Faserdehnung während des zusammen ziehen der Stämmlinge zunimmt, dann im Moment des Realises plötzlich abfällt, um in Folge harmonisch gedämpft auszuschwingen (nährungsweise).

In [ ]:
ptq_series.plot_measurement_sensors(sensor_names=elasto_names)

### PTQ: Bestimmung von Schwingungsparametern

#### PTQ: Selektieren der relevanten Bereiche

Selektiere die Bereiche nach dem Release, bei dem es zu einer harmonisch gedämpften Schwingung kommt. Die Methode Series.get_oscillations sucht im Standardfall nach einem Bereich in den Messdaten mit einer Länge von 20 Sekunden. Der Anfangszeitpunkt wird durch einen plötzlichen Abfall der Dehnung auf unter Null bestimmt, bei dem die Steigung mindestens -25 beträgt. Die Suche nach dem Startzeitpunkt beginnt erst 60 Sekunden nach Messungsbeginn. Der entsprechende Code befindet sich im Paket classes/measurement.py und utils/select_oscillation.py. Die so isolierten Bereiche werden als Instanzen der Klasse Oscillation initialisiert. Weitere Parameter wie Amplitude, Frequenz und Dämpfung werden direkt berechnet.
Parameter:
- sensor_names: Eine Liste der Sensornamen, für die die Schwingungsdaten identifiziert werden sollen.
- min_time_default: Die Mindestzeitspanne nach Beginn der Messung, nach der die Suche nach Schwingungen beginnt (Standard: 60 Sekunden).
- min_value: Der minimale Wertschwellenwert, damit Sensordaten als gültig betrachtet werden.
- threshold_slope: Der Steigungsschwellenwert, um den Beginn einer Schwingung zu bestimmen.
- duration: Die Dauer, für die die Schwingungsdaten extrahiert werden sollen.

In [ ]:
ptq_series.get_oscillations(
    sensor_names=elasto_names,
    min_time_default=60,
    min_value=50,
    threshold_slope=-50,
    duration=17.5
)

#### PTQ: Selektion optisch prüfen in Plots

Plotten der relevanten Sensoren bzw. der selektierten Bereiche. Die Plots werden im Verzeichnis ptq/plots/select_oscillations_single/ bzw. ptq/plots/select_oscillations_combined/ gespeichert. In einem Combined-Plot werden alle 4 Elastometer einer Messung gemeinsam dargestellt. Hier wird manuell anhand der Plots geprüft, ob für alle Messungen und Sensoren der richtige Bereich ausgewählt wurde.

In [ ]:
ptq_series.plot_oscillations_for_measurements(sensor_names=elasto_names, combined=False)
ptq_series.plot_oscillations_for_measurements(sensor_names=elasto_names, combined=True)

#### PTQ: Anpassen der harmonisch gedämpften Schwingung

Aus der PTQ-Messreihe 'ptq_series' wird über `get_oscillations_list` für alle Messungen, getrennt für jeden Sensor (Elastometer), die `Oscillation`-Instanz in eine Liste zusammengeführt. Entsprechend gibt es für jede PTQ-Messung 4 `Oscillation`-Instanzen (für die 4 Elastometer).

`oscillation.fit` passt alle Schwingungsdaten mit einer allgemeinen Funktion für harmonisch gedämpfte Schwingungen an:
\[
y(t) = A \cdot e^{-\delta t} \cdot \cos(\omega_d \cdot t + \phi) + y_0
\]

##### Parameterbeschreibung:
- `A` (Anfangsamplitude): Der Anfangswert der Amplitude der Schwingung. Dieser Parameter bestimmt die initiale Höhe der Schwingungsamplitude.
- `δ` (Dämpfungskoeffizient): Dieser Wert bestimmt, wie schnell die Amplitude der Schwingung mit der Zeit abnimmt. Ein höherer Wert führt zu einer schnelleren Dämpfung der Schwingung.
- `ω_d` (gedämpfte Kreisfrequenz): Die Frequenz der gedämpften Schwingung in Radiant pro Sekunde. Dieser Parameter bestimmt, wie schnell die Schwingung oszilliert.
- `φ` (Phasenwinkel): Der Anfangsphasenwinkel der Schwingung. Dieser Wert bestimmt den Startpunkt der Schwingung im Schwingungszyklus.
- `y_0` (Vertikale Verschiebung): Dieser Parameter verschiebt die gesamte Schwingungskurve vertikal und ermöglicht es, die Schwingung an die mittlere Position der Daten anzupassen.
- `t_0` (Horizontale Verschiebung): Dieser Parameter verschiebt die gesamte Schwingungskurve horizontal über die Zeit und ermöglicht es, die Schwingung an den spezifischen Startpunkt der gemessenen Schwingung anzupassen.

(siehe `ptq/analyse/fitting_function.py`)

### Zusätzliche Parameter und Konfigurationen:
- **Startwerte und Grenzwerte:** Für die Optimierung der Parameter in `scipy.curve_fit` werden Startwerte und Grenzwerte für jeden Parameter übergeben (in `ptq/config.py` definiert).
- **Qualitätsmetriken:** Zur Bewertung der Anpassungsgüte werden Metriken wie MAE (mittlerer absoluter Fehler), RMSE (Root Mean Square Error), und \( R^2 \) (Bestimmtheitsmaß) berechnet. Zusätzlich werden normalisierte Varianten (NRMSE und NMAE) zur besseren Vergleichbarkeit verwendet.
- **Warnungen bei Überschreitung der Grenzwerte:** Wenn die für eine Metrik definierten Grenzwerte überschritten werden, wird eine Warnung im Log-Protokoll vermerkt, um auf mögliche Probleme bei der Anpassung hinzuweisen (in `ptq/config.py` definiert). Auf Basis dieser Warnung können:
  - Start- und Grenzwerte sowie die Methodik angepasst werden.
  - Betroffene Datensätze später ausgeschlossen werden, um fehlerhafte Anpassungen zu vermeiden.
- **Interpolation:** Diese Option aktiviert die Interpolation der Datenpunkte, um eine ausreichende Dichte für `curve_fit` zu gewährleisten. Hierbei wird `scipy.interpolate.PchipInterpolator` verwendet, um Über- und Unterschwingungen, die nicht in den Originaldaten vorhanden sind, zu vermeiden. Nach optischer Prüfung zeigte diese Methode die besten Ergebnisse.

##### Visualisierungsoptionen:
- **Plot:** Wenn auf `True` gesetzt, wird für jede Oscillation ein Plot der angepassten Funktion zusammen mit den Originaldaten erstellt und in `working_dir/PTQ/plots/` gespeichert.
- **Plot-Fehlerverteilung:** Wenn `plot_error` auf `True` gesetzt ist, wird ein Histogramm der Fehlerverteilung (Residuen) für jeden Fit erstellt und ebenfalls in `working_dir/PTQ/plots/` gespeichert.

In [ ]:
ptq_oscillations_ls = ptq_series.get_oscillations_list()

initial_param = {
    "initial_amplitude": 170,
    "damping_coeff": 0.32,
    "frequency": 0.44,
    "phase_angle": 0,
    "y_shift": 0,
    "x_shift": 0
}

param_bounds = {
    "initial_amplitude": (150, 250),
    "damping_coeff": (0.1, 1),
    "frequency": (0.35, 0.58),
    "phase_angle": (-0.2, 0.2),
    "y_shift": (-60, 60),
    "x_shift": (-0.25, 0.75),
}

metrics_warning = {
    "pearson_r": (0.75, 1),
    "nrmse": (0, np.inf),
    "mae": (0, np.inf),
    "nmae": (0, 0.10)
}

for oscillation in ptq_oscillations_ls:
    oscillation.fit(initial_param, param_bounds, optimize_criterion="mae", metrics_warning=metrics_warning, plot=True,
                    plot_error=True, dir_add="", interpolate=True)

#### Fehlerverteilung der Funktionsanpassung an Messdaten

Die Funktion sammelt für alle Oscillation-Objekte die Fehler-Arrays der Anpassung und normalisiert die Fehler (um Unterschiede in der Skalierung zu entfernen).
Anschließend werden die Fehler für alle Messungen A) für alle Sensoren gemeinsam und B) getrennt für jeden Sensor geplotet. Es werden Q-Q-Plot, Violin-Plot und Histogramme für den gleichen Sachverhalt erstellt und in `working_directory\PTQ\plots\series_osc_errors` abgelegt.

- `trim_hist_percent`: Beschneidet die Daten Links und Rechts um die äußersten x Prozent, da die Verteilung im Zentrum sonst kaum zu bewerten ist. Wirkt sich nur auf die Histogramme aus.

In [ ]:
all_normalized_errors = ptq_series.plot_osc_errors(plot_qq=True, plot_violin=True, plot_hist=True, hist_trim_percent=5)

#### PTQ: Zusammenfassung der Schwingungsparameter aller Oscillations als DataFrame

'ptq_series.get_oscillations_df' fasst aus allen Oscillation-Instanzen der Messreihe ('ptq_series') die Schwingungsparameter als pandas.DataFrame zusammen

In [ ]:
ptq_metadata_df = ptq_series.get_oscillations_df()
# Converting 'sensor_name' to categorical
ptq_metadata_df['sensor_name'] = ptq_metadata_df['sensor_name'].astype('category')

metrics_warning_count = ptq_metadata_df['metrics_warning'].sum()
print(f"metrics_warning_count: {metrics_warning_count}")
ptq_metadata_df.head(3)

#### PTQ: Berechnung ergänzender Schwingungsparameter

Ergänze rechnerische Schwingungsparameter die Rust(2013) genutzt hat

In [ ]:
def calculate_additional_parameters(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate additional parameters (ungedämpfte Frequenz, Dämpfungsgrad)
    and append them to the DataFrame.

    Args:
    df (pd.DataFrame): Input DataFrame with necessary columns, where 'frequency' is in Hz.

    Returns:
    pd.DataFrame: DataFrame with additional parameters added.
    """
    # Erstelle eine Kopie des DataFrames, um inplace-Operationen zu vermeiden
    df = df.copy()

    # Umbenennen der Spalte 'frequency' zu 'frequency_damped'
    df = df.rename(columns={'frequency': 'frequency_damped'})

    # Berechnung der ungedämpften Frequenz
    df['frequency_undamped'] = np.sqrt(df['frequency_damped']**2 + (df['damping_coeff'] / (2 * np.pi))**2)

    # Berechnung des Dämpfungsgrades
    df['damping_ratio'] = df['damping_coeff'] / df['frequency_damped']

    return df

# Add additional parameters
ptq_metadata_df = calculate_additional_parameters(ptq_metadata_df)
ptq_metadata_df.head(5)


### PTQ: Daten Vorbereitung (Data Cleaning)

Ergänze maximale Dehnung und Stauchung zum ptq_metadata_df

In [ ]:
# Merging ptq_metadata_df and ptq_df_elasto
ptq_metadata_df = pd.merge(ptq_metadata_df, ptq_df_elasto, on=['id', 'sensor_name'], how='left')
ptq_metadata_df

In [ ]:
# Liste der Spalten, die beibehalten werden sollen
select_cols = [
    'id', 
    'sensor_name',
    'max_strain',
    'max_compression',
    'm_amplitude', 
    'm_amplitude_2', 
    'initial_amplitude',
    'damping_coeff', 
    'damping_ratio', 
    'frequency_damped', 
    'frequency_undamped', 
    'y_shift',
    'pearson_r',
    'nrmse', 
    'nmae', 
]

# DataFrame auf die gewünschten Spalten reduzieren
ptq_metadata_df = ptq_metadata_df[select_cols]


In [ ]:
ptq_metadata_df.head(3)

### LS3: Daten Vorbereitung (Data Cleaning)
Definition relevanter Spalten aus ls3_metadata_df und Anpassung der Datenstruktur, um nur die erforderlichen Variablen abzufragen. 

In [ ]:
# Selecting relevant columns and creating a copy to avoid SettingWithCopyWarning
select_cols = ['sensor_id', 'measurement_id', 'max', 'release']
ls3_metadata_df = ls3_metadata_df[select_cols].copy()

# Extracting values from tuples in 'max'
ls3_metadata_df['max'] = ls3_metadata_df['max'].apply(lambda x: x[1] if isinstance(x, tuple) else x)
ls3_metadata_df.head(3)
ls3_metadata_df

## Combined: Zusammenführen der Daten von LS3, PTQ und Versuchsprotokoll (Merge)

Führe Metadaten des Versuchsablaufes zusammen mit Daten der Elastometer (Schwingungsparameter und Metadaten)

In [ ]:
df = pd.merge(ptq_metadata_df, series_df, on='id', how='left')

Ergänze die Daten der Sensorpositionierung

In [ ]:
# Merging df and elasto_df
# Geräte vom Typ Elastometer auswählen aus dem sensor_df
elasto_df = sensor_df[sensor_df["type"] == "Elasto"].copy()

# Passe die sensor_namen einheitlich für beide DataFrames an
elasto_df["sensor_name"] = elasto_df["type"].astype(str) + "(" + elasto_df["sensor_id"].astype(str) + ")"

# Wähle nur die relevanten spalten aus
select_cols = ["sensor_name", "location", "height", "diameter", "direction"]
elasto_df = elasto_df[select_cols]

# Perform the left join on sensor_name
df = df.merge(elasto_df, on="sensor_name", how="left")

# Convert sensor_name back to a categorical type
df["sensor_name"] = df["sensor_name"].astype("category")

Ergänze die Metadaten der Kraftmessdosen (LS3), hier zwei Kraftmessdosen mit verschiedener Aussage und jeweils für jede Messung aktiv

In [ ]:
# Function to prepare ls3_metadata
def prepare_ls3_metadata(df, sensor_id, new_prefix):
    return (df.query(f"sensor_id == '{sensor_id}'")
            .drop('sensor_id', axis=1)
            .add_prefix(new_prefix)
            .rename(columns={f'{new_prefix}measurement_id': sensor_id}))

# Merging ls3_metadata with the main DataFrame
df = df.merge(prepare_ls3_metadata(ls3_metadata_df, '14:BF:E6', 'rope_'), on='14:BF:E6', how='left')     
df = df.merge(prepare_ls3_metadata(ls3_metadata_df, '14:99:1E', 'cable_'), on='14:99:1E', how='left')     
df.head(3)

## Combined: Daten Vorbereitung (Data Cleaning)

Filterung und Kategorisierung der Daten für weitergehende Analysen.

### Combined: Filtern der relevanten Messungen aus Versuchsprotokoll

Messung id = 1 war eine Vorabtest des Versuchsaufbaus und wurde mit einer unregelmäßigen Vorspannung von 2.5 durchgeführt.
Messung id = 29 war der einzige Versuch mit der Behandlung cobra_static_slack.
Die entsprechenden 2 Messungen werden entfernt.

In [ ]:
# filtering and cleaning 
df = (df.drop(['14:BF:E6', '14:99:1E', 'cable_release', 'rope_max', 'time'], axis=1)
      .query("release_force_target in [2.0, 2.4, 2.8] and treatment in ['free', 'gefa_dynamic', 'cobra_static']"))

In [ ]:
# Nicht mehr genutzte Kategorien entfernen
df["treatment"] = df["treatment"].cat.remove_unused_categories()

In [ ]:
df.describe()

#### Datendokumentation:

In [ ]:
# Lade das Dictionary mit der Daten Dokumentation
with open(data_path / "feature_data_dict.json", "r", encoding="utf-8") as f:
    feature_data_dict = json.load(f)

data_dict = {**sensor_position_dict, **series_dict, **feature_data_dict}

# Keys aller gewünschten Variablen (aus kombiniertem Dict)
selected_keys = ["id", "treatment", "sensor_name", "location", "height", "diameter", "rope_release", "max_strain", "calc_max_strain", "strain_difference", "release_force_target", "cable_max", "max_compression", "m_amplitude", "m_amplitude_2", "initial_amplitude", "damping_coeff", "damping_ratio", "frequency_damped", "frequency_undamped", "y_shift", "pearson_r", "nmae"]

# Nutze bestehende Felder
selected_fields = ["Zeichen", "Variable", "Deutsch", "Einheit", "Beschreibung"]

# Erzeuge DataFrame für LaTeX mit einheitlichem Aufbau
combined_dict_df = extract_latex_dict_from_json(data_dict, selected_keys, selected_fields)

# Exportiere als LaTeX
latex_string = combined_dict_df.to_latex(index=False, escape=False)

caption = "Feldversuch 2 - Ergebnisse, Daten Dokumentation"
caption_long = "Feldversuch 2 - Ergebnisse, Kräfte, Dehnungen und Schwingungsparameter, Daten Dokumentation"

save_latex_table(latex_string, caption, latex_export_directory, caption_long=caption_long)

Ausgabe für CHAT-GPT import

In [ ]:
df.to_csv(working_directory / 'export_data/2023_Kronensicherung_Plesse_Kraefte_Schwingungen_Full_Dataset.bz2', index=False)

In [ ]:
# Speichern
data_dict_path = working_directory / 'export_data/2023_Kronensicherung_Plesse_Kraefte_Schwingungen_Full_Data_Dict.json'
with open(data_dict_path, "w", encoding="utf-8") as f:
    json.dump(data_dict, f, indent=2, ensure_ascii=False)

In [ ]:
#df.dtypes

## Combined: Analyse der zusammengeführten Daten für LS3, PTQ und Versuchsprotkoll

### Combined: Explorative Datenanalyse

In [ ]:
df.head(3)

In [ ]:
df.columns

### Combined: Definition von Darstellungsstandards
Festlegen von Farbcodes für einheitliche Darstellung von Sensoren und Behandlungsvarianten für alle nachfolgenden Plots.

In [ ]:
PLOT_MANAGER = ptq_series.PLOT_MANAGER

# Seaborn "deep" Palette holen
deep_palette = sns.color_palette("deep", 6)

# Zuweisen von Farben aus der "deep" Palette an die Sensoren
sensor_color_dict = {sensor: color for sensor, color in zip(elasto_names, deep_palette[:len(elasto_names)])}

# Zuweisen von Farben aus der "deep" Palette an die Treatments
treatment_color_dict = {treatment: color for treatment, color in zip(treatment_order, deep_palette[:len(treatment_order)])}


Plotting Tite-, Achsen-, Filenamen erzeugen aus Data-Dict

In [ ]:
def get_axis_label(key: str, data_dict: Dict[str, Dict[str, str]]) -> str:
    return f"{data_dict[key]['Deutsch']} {data_dict[key]['Zeichen']} [{data_dict[key]['Einheit']}]"

def get_plot_title(x_key: str, y_key: str, data_dict: Dict[str, Dict[str, str]], prefix: str = "Regression") -> str:
    return (
        f"{prefix}: {data_dict[y_key]['Deutsch']} "
        f"({data_dict[y_key]['Zeichen']}) vs. "
        f"{data_dict[x_key]['Deutsch']} "
        f"({data_dict[x_key]['Zeichen']})"
    )

def get_legend_title(key: str, data_dict: Dict[str, Dict[str, str]]) -> str:
    return data_dict[key]["Deutsch"]

def get_filename(x_key: str, y_key: str, prefix: str) -> str:
    return slugify(f"{prefix}_{y_key}_vs_{x_key}", separator="_")

### LS3: Analyse Spitzenlasten in der KS

Analyse der Spitzenlasten in der KS gruppiert nach Ziel-Vorspannung und Treatment.

In [ ]:
# Filtern und Umbenennen
filtered_df = (
    df[df['treatment'].isin(['gefa_dynamic', 'cobra_static'])]
    .rename(columns={
        'release_force_target': 'Ziel-Vorspannung',
        'treatment': 'Behandlungsvariante',
        'cable_max': 'Kraftspitze KS'
    })[['Behandlungsvariante', 'Ziel-Vorspannung', 'Kraftspitze KS']]
)

In [ ]:

# Gruppieren, Aggregieren und Pivotieren
pivoted_values = (
    filtered_df
    .groupby(['Ziel-Vorspannung', 'Behandlungsvariante'], observed=True)['Kraftspitze KS']
    .agg(['min', 'mean', 'max'])
    .unstack(level=0)
    .swaplevel(axis=1)
    .sort_index(axis=1, level=[0, 1], ascending=[True, False])
)
# Begrenze die Werte im Index Level 0 auf zwei Nachkommastellen
pivoted_values.columns = pd.MultiIndex.from_tuples(
    [
        (f"{level_0:.2f}", level_1) if isinstance(level_0, float) else (level_0, level_1)
        for level_0, level_1 in pivoted_values.columns
    ],
    names=pivoted_values.columns.names
)

# Ergebnis anzeigen
pivoted_values

In [ ]:
latex_string = pivoted_values.to_latex(index=True, escape=True, float_format="{:0.2f}".format, column_format="l|rrr|rrr|rrr", multicolumn=True,
    multicolumn_format="c")

caption = "Feldversuch 2 - Ergebnisse, Spitzenlasten in der KS"
caption_long = "Feldversuch 2 - Spitzenlasten in der KS gruppiert über Ziel-Vorspannung und Behandlungsvariante, angegeben ist jeweils pro Gruppe das Minimum, der Mittelwert und das Maximum, die Variante 'free' ist nicht aufgeführt, da hier keine KS eingesetzt wurde, alle Werte in kN"

save_latex_table(latex_string, caption, latex_export_directory, caption_long)

In [ ]:
# Filtern der Einträge mit dem maximalen Wert in 'cable_max'
max_cable_max = df['cable_max'].max()
filtered_df = df[df['cable_max'] == max_cable_max]

# Innerhalb der gefilterten Einträge den maximalen 'max_strain' finden
max_value_row = filtered_df.loc[filtered_df['max_strain'].idxmax()]
#max_value_row

In [ ]:
# Liste der gewünschten Spalten in der gewünschten Reihenfolge
columns_to_display = [
    'id', 'treatment', 'release_force_target', 'rope_release', 
    'cable_max', 'sensor_name', 'max_strain', 'max_compression'
]

# Zeile auf die gewünschten Spalten in der angegebenen Reihenfolge beschränken
max_value_row_filtered = max_value_row[columns_to_display]

max_value_row_filtered['treatment'] = slugify(max_value_row_filtered['treatment'])

# Erstelle ein DataFrame mit den zusätzlichen Informationen aus `data_dict`
expanded_data = []
for col in columns_to_display:
    expanded_data.append({
        "Zeichen": data_dict[col]["Zeichen"],
        "Deutsch": data_dict[col]["Deutsch"],
        "Wert": max_value_row_filtered[col],
        "Einheit": data_dict[col]["Einheit"],
    })

# Neues DataFrame erstellen
expanded_df = pd.DataFrame(expanded_data)

# LaTeX-String erstellen
latex_string = expanded_df.to_latex(
    index=False, 
    escape=False, 
    column_format="llrr",  # Spaltenformat angepasst
    float_format="{:0.2f}".format
)

# LaTeX-Tabelle mit Beschriftung versehen
caption = "Feldversuch 2 - Ergebnisse, Werte der Messung mit Spitzenlast"
caption_long = "Feldversuch 2 - Ergebnisse, Messung mit der höchsten Last in der KS (nur maximale Messwerte des Elastometers mit der höchsten Faserdehnung)"

# Funktion zum Speichern aufrufen (angenommen save_latex_table ist definiert)
save_latex_table(latex_string, caption, latex_export_directory, caption_long)

# Ergebnis anzeigen
expanded_df


### Combined: Vergleichende Berechnung der Randfaserdehnung

In [ ]:
# Funktion zur Berechnung der Randfaserdehnung (in Dezimalschreibweise)
def calculate_epsilon(force: float, H: float, h: float, d: float, E: float) -> float:
    """
    Berechnet die Randfaserdehnung epsilon (dimensionslos, nicht in %).
    
    Parameters:
        force (float): Zugkraft im Seil (in kN)
        H (float): Höhe des Angriffspunktes der Zugkraft über dem Stammfuß (in m)
        h (float): Höhe des Berechnungspunktes über dem Boden (in m)
        d (float): Durchmesser des Stammquerschnitts auf Höhe h (in m)
        E (float): Elastizitätsmodul des Holzes (in MPa)
    
    Returns:
        float: Berechnete Randfaserdehnung epsilon (dimensionslos)
    """
    force_N = force * 1000  # Umrechnung der Zugkraft von kN in N
    M = force_N * (H - h)  # Biegemoment in N·m
    y_max = d / 2  # maximaler Abstand zur neutralen Achse in m
    I = (np.pi / 64) * d**4  # Flächenträgheitsmoment in m^4
    E_Pa = E * 1e6  # Umrechnung des Elastizitätsmoduls von MPa in N/m^2
    epsilon = (M * y_max) / (E_Pa * I)  # Dehnung in Dezimalschreibweise
    return epsilon

# Funktion zur Berechnung der absoluten Längenänderung
def calculate_delta_l(epsilon: float, l0: float) -> float:
    """
    Berechnet die absolute Längenänderung Delta L in Mikrometer.
    
    Parameters:
        epsilon (float): Relative Dehnung (dimensionslos)
        l0 (float): Ausgangslänge des Elastometers (in mm)
    
    Returns:
        float: Absolute Längenänderung Delta L in Mikrometer (µm)
    """
    l0_m = l0 / 1000  # Umrechnung der Ausgangslänge von mm in m
    delta_L = epsilon * l0_m * 1e6  # Umrechnung der Längenänderung in µm
    return delta_L


In [ ]:
# Fixwerte
l0 = 200  # mm, Ausgangslänge Elastometer
E = 8500  # Elastizitätsmodul Buche in MPa
height_rope = 18.30  # m, Höhe des Angriffspunktes der Zugkraft

# Berechnung der neuen Spalten
df['calc_max_strain_relativ'] = df.apply(lambda row: calculate_epsilon(
    row['rope_release'], height_rope, row['height'], row['diameter'], E), axis=1)
df['calc_max_strain'] = df['calc_max_strain_relativ'].apply(lambda epsilon: calculate_delta_l(epsilon, l0))

In [ ]:
# Berechnung der Differenz und der absoluten Differenz in Prozent
df['strain_difference'] = (df['calc_max_strain'] - df['max_strain']) / df['max_strain'] * 100

In [ ]:
df.head(4)

In [ ]:
# Grouping DataFrame by 'ptq_sensor_name'
grouped = df.groupby('sensor_name')

# Perform linear regression and plot for each group
for name, group in grouped:
    # Linear Regression with statsmodels
    X = sm.add_constant(group['max_strain'])  # Adding a constant for the intercept
    y = group['calc_max_strain']
    model = sm.OLS(y, X).fit()
    # Print the summary of the linear regression model
    print(f"Linear Regression Summary for {name}:\n")
    print(model.summary())

In [ ]:
# Achsenvariablen definieren
x_key = 'max_strain'
y_key = 'calc_max_strain'

# Dynamische Labels und Titel erzeugen
x_label = get_axis_label(x_key, data_dict)
y_label = get_axis_label(y_key, data_dict)
plot_title = get_plot_title(x_key, y_key, data_dict, prefix="Regression")
filename = get_filename(x_key, y_key, prefix="regression")
legend_title = get_legend_title("sensor_name", data_dict)

# Plot erstellen
fig = plt.figure(figsize=(10, 6))

for name, group in df.groupby('sensor_name', observed=False):
    sns.regplot(
        x=group[x_key],
        y=group[y_key],
        color=sensor_color_dict.get(name, "gray"),
        label=name,
        scatter_kws={"s": 40}
    )

plt.title(plot_title)
plt.xlabel(x_label)
plt.ylabel(y_label)
plt.legend(title=legend_title)
plt.tight_layout()
plt.show()

# Plot speichern
PLOT_MANAGER.save_plot(fig, filename=filename, subdir="measured_vs_calc_strain")


In [ ]:
# Berechnung von Mittelwerten und Standardabweichungen
df_strain_stats = df.groupby('sensor_name').agg(
    calc_max_strain_mean=('calc_max_strain', 'mean'),
    calc_max_strain_sd=('calc_max_strain', 'std'),
    ptq_max_strain_mean=('max_strain', 'mean'),
    ptq_max_strain_sd=('max_strain', 'std'),
    strain_difference_mean=('strain_difference', 'mean'),
    strain_difference_sd=('strain_difference', 'std')
)
# Automatische Umbenennung der Spalten basierend auf data_dict
columns_new = [(data_dict[var]['Zeichen'], stat) for var, stat in zip(
    ['calc_max_strain', 'calc_max_strain', 'max_strain', 'max_strain', 'strain_difference', 'strain_difference'],
    ['mean', 'sd', 'mean', 'sd', 'mean', 'sd']
)]
df_strain_stats.columns = pd.MultiIndex.from_tuples(columns_new)
df_strain_stats

In [ ]:
# Berechnung des MAPE
strain_difference_mape = df['strain_difference'].abs().mean().round(2)
strain_difference_mape

In [ ]:
# Automatische Einheitenzuweisung
df_units = pd.DataFrame([[data_dict[var]['Einheit'] for var in ['calc_max_strain', 'calc_max_strain', 'max_strain', 'max_strain', 'strain_difference', 'strain_difference']]],
                        columns=df_strain_stats.columns, index=['Einheit'])

# MAPE als eigene Zeile hinzufügen
df_mape = pd.DataFrame([['', '', '', '', strain_difference_mape, '']], 
                        columns=df_strain_stats.columns, index=['MAPE'])

# DataFrames zusammenführen
df_strain_stats_add = pd.concat([df_units, df_strain_stats, df_mape])
df_strain_stats_add

In [ ]:
# Konvertierung des DataFrames mit Multi-Index-Spalten in einen LaTeX-String
latex_string = df_strain_stats_add.to_latex(
    index=True,
    escape=False,
    float_format="{:0.2f}".format,
    multicolumn=True,
    multicolumn_format="c",
    column_format="l|rr|rr|rr"
)

# Definition der Beschriftung für die LaTeX-Tabelle
caption = "Feldversuch 2 - Ergebnisse, Zusammenfassung Vergleich gemessene und rechnerische Faserdehnung "
caption_long = "Feldversuch 2 - Ergebnisse, Zusammenfassung Vergleich gemessene und rechnerische Faserdehnung, Mittelwerte und Standardabweichungen der Abweichung der rechnerischen von der gemessenen maximalen Dehnung, gruppiert über Elastometer bzw. Position"

save_latex_table(latex_string, caption, latex_export_directory, caption_long)

In [ ]:
# Create violin plot for strain_difference grouped by sensor_name
fig = plt.figure(figsize=(10, 6))
sns.violinplot(x='sensor_name', y='strain_difference', hue='sensor_name', palette=sensor_color_dict, data=df)
plt.title('Difference between measured and calculated strain')
plt.xlabel('PTQ Sensor Name / Position')
plt.ylabel('Strain Difference (%)')
# Layout anpassen und Plot anzeigen
plt.tight_layout()
plt.show()

PLOT_MANAGER.save_plot(fig, filename="diff_measured_vs_calc_strain", subdir="measured_vs_calc_strain")

### Theoretische Berechnungen der Belastung der KS

In [ ]:
def calculate_force(epsilon: float, H: float, h: float, d: float, E: float) -> float:
    """
    Berechnet die Zugkraft force (in kN) basierend auf der Randfaserdehnung epsilon bzw. der Dehnung an der Elastizitätsgrenze.
    
    Parameters:
        epsilon (float): Randfaserdehnung (dimensionslos, nicht in %)
        H (float): Höhe des Angriffspunktes der Zugkraft über dem Stammfuß (m)
        h (float): Höhe des Berechnungspunktes über dem Boden (m)
        d (float): Durchmesser des Stammquerschnitts auf Höhe h (m)
        E (float): Elastizitätsmodul des Holzes (in MPa)
    
    Returns:
        float: Berechnete Zugkraft force (in kN)
    """
    y_max = d / 2  # maximaler Abstand zur neutralen Achse in m
    I = (np.pi / 64) * d**4  # Flächenträgheitsmoment in m^4
    E_Pa = E * 1e6  # Umrechnung des Elastizitätsmoduls von MPa in N/m^2
    force_N = (epsilon * E_Pa * I) / ((H - h) * y_max)  # Zugkraft in N
    force_kN = force_N / 1000  # Umrechnung in kN
    return force_kN

In [ ]:
# Fixwerte
epsilon = 0.0026  # Elastizitätsgrenze Buche
E = 8500  # Elastizitätsmodul Buche in MPa
height_ks = 17.40  # m, Höhe des Angriffspunktes der KS
height_forke = 10.31  # m, Höhe des Zwiesels
stem_diameter = 0.35  # m, Durchmesser des Stammes knapp über dem Zwiesel

# Berechnung der Zugkraft
force = calculate_force(epsilon, height_ks, height_forke, stem_diameter, E)
force

In [ ]:
tree_height = (26.15 + 27.2)/2
optimal_ks_height = ((tree_height - height_forke) * (2/3)) + height_forke
optimal_ks_height

In [ ]:
force = calculate_force(epsilon, optimal_ks_height, height_forke, stem_diameter, E)
force

### PTQ: Analyse der Dehnungswerte nach Elastometer und Behandlungsvarianten

Plotten der Maximalwerte der Vorspannung für Dehnung und Stauchung. Für jede Messung werden erst die maximalen Werte berechnet und dann getrennt nach Elasto die Verteilung im Boxplot dargestellt.
Auffällig ist Elasto(90) mit den höchsten Dehnungswerten. Dieses Gerät ist am geringfügig dünneren Stämmling angebracht. Die Vorspannung wurde aufgrund der maximalen Messwerte von 429 µm nicht weiter erhöht.

In [ ]:
# Erstelle die Subplots
fig, axs = plt.subplots(1, 2, figsize=(8, 4))

# Boxplot für die minimalen Werte (maximale Compression)
sns.boxplot(data=df, x='sensor_name', y='max_compression', hue='sensor_name', palette=sensor_color_dict, ax=axs[0], legend=False)
axs[0].set_title("Maximum Compression per Elasto")
axs[0].set_xlabel("Elasto Name")
axs[0].set_ylabel("fiber compression [µm]")

# Boxplot für die maximalen Werte (maximale Strain)
sns.boxplot(data=df, x='sensor_name', y='max_strain', hue='sensor_name', palette=sensor_color_dict, ax=axs[1], legend=False)
axs[1].set_title("Maximum Strain per Elasto")
axs[1].set_xlabel("Elasto Name")
axs[1].set_ylabel("fiber strain [µm]")

# Layout anpassen und Plot anzeigen
plt.tight_layout()
plt.show()

PLOT_MANAGER.save_plot(fig, filename="strain_max_per_elasto", subdir="strain_max_per_elasto")

In [ ]:
# # Mixed-Effects Model
# model = smf.mixedlm(
#     "max_compression ~ C(treatment) + C(sensor_name) + rope_release",
#     data=df,
#     groups=df["id"]  # Gruppierung nach Beobachtungsgruppe
# ).fit()
# 
# # Ergebnisse
# print(model.summary())


In [ ]:
# # Tukey HSD Test für die 'treatment' Gruppe
# tukey_treatment = mc.pairwise_tukeyhsd(endog=df['max_compression'],
#                                     groups=df['treatment'],
#                                     alpha=0.05)
# print(tukey_treatment)
# 
# # Tukey HSD Test für die 'sensor_name' Gruppe
# tukey_sensor = mc.pairwise_tukeyhsd(endog=df['max_compression'],
#                                  groups=df['sensor_name'],
#                                  alpha=0.05)
# print(tukey_sensor)

### Combined: Latex-Export von Daten für Anhang

In [ ]:
def generate_grouped_latex_tables(df_latex: pd.DataFrame, caption: str, column_format: str, group_by: str, latex_export_directory: Path) -> None:
    """
    Generate grouped LaTeX tables for each unique value in a specified column of a DataFrame.

    Parameters:
        df_latex (pd.DataFrame): The DataFrame with already formatted columns.
        caption (str): The caption for the LaTeX tables.
        column_format (str): The format for the LaTeX tables.
        group_by (str): The column name to group by.
        latex_export_directory (Path): The directory to save the LaTeX tables.

    Returns:
        None
    """
    try:
        # DataFrame grouped by specified column
        grouped = df_latex.groupby(group_by, observed=True)

        # LaTeX tables to be combined in a single file
        combined_tables = []

        for group, group_df in grouped:
            # Drop the group_by column
            group_df = group_df.drop(columns=[group_by])

            # Format 'id' column as string if it exists
            if 'id' in group_df.columns:
                group_df['id'] = group_df['id'].astype(str)

            # Calculate statistics
            mean_row = group_df.mean(numeric_only=True).rename('Mean')
            median_row = group_df.median(numeric_only=True).rename('Median')
            sd_row = group_df.std(numeric_only=True).rename('SD')

            # Combine stats with original DataFrame
            stats_df = pd.concat([group_df, mean_row.to_frame().T, median_row.to_frame().T, sd_row.to_frame().T])

            # Generate LaTeX string from DataFrame
            df_latex_string = stats_df.to_latex(
                index=True,
                escape=False,
                column_format=column_format,
                float_format="{:0.2f}".format
            )

            # Create caption and label for the group
            caption_text = create_caption(caption, f"{caption} für {slugify(group, separator=' ')}")
            label_clean = create_label(caption=caption, additional_label=group)

            # Generate LaTeX table
            latex_table = generate_latex_table(df_latex_string, caption_text, label_clean)

            combined_tables.append(latex_table)

        # Combine all tables and save to a single file
        if combined_tables:
            final_output = "\n\n".join(combined_tables)
            file_name = create_label(caption) + ".tex"
            save_to_file(final_output, latex_export_directory / file_name)

    except Exception as e:
        print(f"Error generating grouped LaTeX tables: {e}")


In [ ]:
# Liste der benötigten Variablen
variables = ["id", "treatment", "sensor_name", "location", "height", "diameter", "rope_release", "max_strain", "calc_max_strain", "strain_difference"]

# DataFrame kopieren und die gewünschten Spalten auswählen
df_latex = df.copy()[variables]

# Spaltennamen mit den Kurzbezeichnungen (Zeichen) aus dem data_dict umbenennen
df_latex = df_latex.rename(columns={var: data_dict[var]["Zeichen"] for var in variables})
# Funktionsaufruf mit Beispielparametern
generate_grouped_latex_tables(
    df_latex=df_latex,
    caption="Feldversuch 2 - Ergebnisse, Vergleich gemessene und rechnerische Faserdehnung",
    column_format="lll|lrrr|rrr",
    group_by="treatment",
    latex_export_directory=latex_export_directory
)


In [ ]:
variables = ['id', 'sensor_name', 'treatment', 'release_force_target', 'rope_release', 'cable_max', 'max_strain', 'max_compression',]

# DataFrame kopieren und die gewünschten Spalten auswählen
df_latex = df.copy()[variables]

# Spaltennamen mit den Kurzbezeichnungen (Zeichen) aus dem data_dict umbenennen
df_latex = df_latex.rename(columns={var: data_dict[var]["Zeichen"] for var in variables})
# Funktionsaufruf mit Beispielparametern
generate_grouped_latex_tables(
    df_latex=df_latex,
    caption="Feldversuch 2 - Ergebnisse, Kräfte und Dehnungen",
    column_format="lrl|rrr|rr",
        group_by="treatment",
    latex_export_directory=latex_export_directory
)


In [ ]:
variables_2 = ['id', 'sensor_name', 'treatment', 'm_amplitude', 'm_amplitude_2', 'initial_amplitude', 'damping_coeff', 'damping_ratio', 'frequency_damped', 'frequency_undamped', 'y_shift', 'pearson_r', 'nmae']

# DataFrame kopieren und die gewünschten Spalten auswählen
df_latex = df.copy()[variables_2]

# Spaltennamen mit den Kurzbezeichnungen (Zeichen) aus dem data_dict umbenennen
df_latex = df_latex.rename(columns={var: data_dict[var]["Zeichen"] for var in variables_2})
# Funktionsaufruf mit Beispielparametern
generate_grouped_latex_tables(
    df_latex=df_latex,
    caption="Feldversuch 2 - Ergebnisse, Schwingungsparameter",
    column_format="lrl|rrr|rr|rr|r|rr",
    group_by="treatment",
    latex_export_directory=latex_export_directory
)

### PTQ: Analyse der Schwingungsparameter

In diesem Abschnitt werden die Schwingungsparameter statistisch ausgewertet. Ziel ist es, den Einfluss verschiedener Behandlungsvarianten (treatment) auf die gemessenen Schwingungsparameter zu untersuchen und dabei auch den potenziellen Einfluss der Vorspannung (rope_release) und Sensorposition (sensor_name) zu berücksichtigen.


In [ ]:
variables = [
    'm_amplitude', 
    'm_amplitude_2',
    'max_strain',
    'max_compression',
    'initial_amplitude',
    'damping_coeff', 
    'damping_ratio', 
    'frequency_damped', 
    'frequency_undamped',
    'pearson_r',
    #'nrmse', 
    'nmae'
]

#### Systematischer Einfluss der Sensorposition

Ziel: Visuell erkennen, ob unterschiedliche Sensoren konsistent andere Werte liefern und ob dieser Effekt die Interpretation der treatment-Effekte erschwert.

In [ ]:
# Plot erstellen
for var in variables:
    fig = plt.figure(figsize=(8, 5))
    sns.boxplot(x="sensor_name", y=var, data=df, hue="treatment", palette=treatment_color_dict,  dodge=True)
    # Stripplot: Punkte zur Veranschaulichung der Verteilung
    sns.stripplot(x="sensor_name", y=var, data=df, hue="treatment", palette=treatment_color_dict, dodge=True, alpha=1, jitter=True, size=5, legend=False)
    # Titel und Achsentitel setzen
    plt.title(f"Einfluss von treatment auf {var} gruppiert über sensor_name")
    plt.xlabel("Sensor Name")
    plt.ylabel(var)
    plt.tight_layout()
    #plt.show()
    PLOT_MANAGER.save_plot(fig, filename=f"effect_sensor_treatment_{var}", subdir="combined/sensor")

#### Systematischer Einfluss der Behandlungsvariante

Ziel: Feststellen, ob die Variation durch unterschiedliche Behandlungen relativ zur sensorbedingten Variation unterscheidbar ist.

In [ ]:
# Plot erstellen
for var in variables:
    fig = plt.figure(figsize=(8, 5))
    sns.boxplot(x="treatment", y=var, data=df, palette=sensor_color_dict, hue='sensor_name')
    sns.stripplot(x="treatment", y=var, data=df, palette=sensor_color_dict, hue='sensor_name', dodge=True, alpha=1, jitter=True, size=5, legend=False)
    plt.title(f"Einfluss von sensor_name auf {var} gruppiert über treatment")
    plt.xlabel("treatment")
    plt.ylabel(var)
    plt.tight_layout()
    #plt.show()
    PLOT_MANAGER.save_plot(fig, filename=f"effect_treatment_sensor_{var}", subdir="combined/treatment")

#### Versuche Mixed-Linear Model
Die Daten sind hierarchisch: Mehrere Messungen (vier Sensoren) pro Beobachtungseinheit (`id`). Ein Mixed-Effects Modell könnte diese Struktur abbilden, indem zufällige Effekte für `id` und feste Effekte für `treatment` sowie `sensor_name` genutzt werden. Zusätzlich könnte `rope_release` als Kovariate eingeführt werden.

Diese Modelle wären theoretisch präziser, aber aufgrund der geringen Stichprobengröße und der komplexen Datenstruktur treten Konvergenzprobleme auf.


In [ ]:
# Ergebnisse als Dictionary speichern
results_dict = {}

for var in variables:
    # Formel für MixedLM: Parameter ~ C(treatment) + C(sensor_name) + rope_release + (1|id)
    formula = f"{var} ~ C(treatment) + C(sensor_name)" #  +  rope_release
    model = smf.mixedlm(formula, data=df, groups=df["id"])
    
    # Modell fitten
    fit = model.fit(reml=True)  # REML ist Standard für gemischte Modelle
    
    # Ergebnisse ausgeben
    print(f"\n### Ergebnisse für {var} ###")
    print(fit.summary())
    
    # Überprüfen, ob das Modell konvergiert ist
    if not fit.converged:
        print("Achtung: Das Modell ist nicht konvergiert. Erwägen Sie Anpassungen (z.B. Skalierung der Daten).")

#### Vereinfachtes Vorgehen durch Aggregation (Mittelung über Sensoren)

Um dennoch aussagekräftige Aussagen zu erhalten, werden die Messungen pro `id` über alle Sensoren gemittelt. Dadurch geht zwar die Variation aufgrund unterschiedlicher Sensoren verloren, aber es entsteht ein stabileres Datenset, in dem jede `id` einen aggregierten Wert pro Parameter hat.

Auf dieser Basis können einfache OLS-Modelle geschätzt werden, z. B. `Parameter ~ C(treatment) + rope_release`. Diese Modelle sind einfacher und sollten stabil konvergieren.

In [ ]:
# Placeholder for results
results = {}

# Step 1: Compute the mean of each variable for each id
grouped_df = df.groupby("id").agg(
    {var: "mean" for var in variables}
).reset_index()

# Add additional variables (e.g., treatment and rope_release) to the aggregated data
grouped_df = grouped_df.merge(
    df[["id", "treatment", "rope_release"]].drop_duplicates(), on="id"
)

# Step 2: Fit linear models to the aggregated data
for var in variables:
    try:
        # Fit a linear model
        model = smf.ols(f"{var} ~ C(treatment) + rope_release", grouped_df).fit()

        # Store relevant results
        results[var] = {
            "p_value_treatment": model.pvalues.get("treatment[T.statisch]", None),
            "p_value_rope_release": model.pvalues.get("rope_release", None),
            "summary": model.summary()
        }
    except Exception as e:
        results[var] = {"error": str(e)}

# Display results
results

Nicht jeder Parameter wird durch `rope_release` beeinflusst. Nur für jene Parameter, bei denen ein signifikanter Einfluss von `rope_release` festgestellt wird, soll dieser Effekt herausgerechnet werden. Auf diese Weise entstehen "bereinigte" Werte, in denen der lineare Einfluss von `rope_release` entfernt ist.

In [ ]:
# Placeholder for results
# Dieses Dictionary wird nun sowohl die Modellobjekte als auch die Kennwerte speichern.
results = {}

# Liste der Variablen, für die rope_release berücksichtigt wird
relevant_vars_rope_release = ['m_amplitude', 'm_amplitude_2', 'max_strain', 'max_compression']

# Step 1: Compute the mean of each variable for each id
grouped_df = df.groupby("id").agg(
    {var: "mean" for var in variables}
).reset_index()

# Add additional variables (e.g., treatment and rope_release) to the aggregated data
grouped_df = grouped_df.merge(
    df[["id", "treatment", "rope_release"]].drop_duplicates(), on="id"
)

# Step 2: Fit linear models to the aggregated data and store model objects and p-Werte in results
for var in variables:
    # Formuliere das Modell dynamisch, abhängig davon, ob rope_release relevant ist
    if var in relevant_vars_rope_release:
        formula = f"{var} ~ C(treatment) + rope_release"
    else:
        formula = f"{var} ~ C(treatment)"

    try:
        # Fit a linear model
        model = smf.ols(formula, grouped_df).fit()

        # Store model and relevant results directly in results
        results[var] = {
            "model": model,
            "p_value_treatment": model.pvalues.get("treatment[T.statisch]", None),
            "p_value_rope_release": model.pvalues.get("rope_release", None) if var in relevant_vars_rope_release else None,
            "summary": model.summary()
        }
    except Exception as e:
        results[var] = {"error": str(e)}

# Display results
print(results)

In [ ]:
# Beispiel für m_amplitude
plt.figure(figsize=(10, 6))
sns.lmplot(data=grouped_df, x='rope_release', y='m_amplitude_2', hue='treatment', ci=95)
plt.title('Einfluss von rope_release auf m_amplitude für verschiedene Treatments')
plt.xlabel('rope_release (kN)')
plt.ylabel('m_amplitude')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(data=grouped_df, x='treatment', y='m_amplitude')
plt.title('Vergleich von m_amplitude zwischen den Treatments')
plt.xlabel('Treatment')
plt.ylabel('m_amplitude')
plt.show()


In [ ]:
import pandas as pd
from tabulate import tabulate
import re

# Liste der Variablen, die in die Tabelle übernommen werden sollen
variables_spec = [
    'm_amplitude_2',
    'max_compression',
    'damping_coeff',
    'damping_ratio',
    'frequency_damped',
    'frequency_undamped',
]

# Umbenennung der Spalten für LaTeX-Notation
column_rename_map = {
    'm_amplitude_2': r'$mA_2$',
    'max_compression': r'$\text{max\_C}$',
    'damping_coeff': r'$\delta$',
    'damping_ratio': r'$D$',
    'frequency_damped': r'$f_d$',
    'frequency_undamped': r'$f_0$',
}

# Funktion zur Erstellung der Modellgüte-Kennzahlen für alle Variablen
def create_model_metrics_table(variables, results):
    metrics_data = {
        "Kennzahl": ["R²", "Adj. R²", "F-St.", "AIC", "N"]
    }

    for var in variables:
        col_name = column_rename_map.get(var, var)  # Verwende gekürzten Namen, falls vorhanden
        if var not in results or 'error' in results[var]:
            metrics_data[col_name] = ["n/a"] * 5
        else:
            model = results[var]['model']
            metrics_data[col_name] = [
                f"{model.rsquared:.4f}",
                f"{model.rsquared_adj:.4f}",
                f"{model.fvalue:.4f}",
                f"{model.aic:.4f}",
                f"{model.nobs:.0f}"
            ]

    # Erstelle die Tabelle mit tabulate
    metrics_df = pd.DataFrame(metrics_data)
    return tabulate(
        metrics_df,
        headers="keys",
        tablefmt="latex_raw",
        floatfmt=".4f",
        showindex=False)

# Funktion zur Erstellung einer LaTeX-Tabelle für die Koeffizienten einer Variable
def create_latex_table_for_variable(var, results):
    if var not in results:
        return f"%% Keine Ergebnisse für Variable {var} vorhanden."
    
    model_result = results[var]
    if 'error' in model_result:
        return f"%% Fehler beim Anpassen des Modells für {var}: {model_result['error']}"
    
    model = model_result['model']
    summary = model.summary2().tables[1]  # Zugriff auf die Tabelle der Koeffizienten

    # Erstelle eine LaTeX-Tabelle mit tabulate für die Koeffizienten
    latex_table = tabulate(
        summary,
        headers=summary.columns,
        tablefmt="latex_booktabs",
        floatfmt=".4f",
        showindex=True
    )

    # Escape Unterstriche in der Variable für LaTeX
    escaped_var = re.sub(r'_', r'\_', var)
    shortened_var = column_rename_map.get(var, escaped_var)

    # Füge die LaTeX-Caption zur Tabelle hinzu
    return f"""
\\begin{{table}}[ht]
    \\centering
    \\caption{{Modellzusammenfassung für {escaped_var} ({shortened_var})}}
    \\begin{{adjustbox}}{{max width=\\textwidth}}
    {latex_table}
    \\end{{adjustbox}}
\\end{{table}}
"""

# Erstelle die Modellgüte-Tabelle für alle Variablen
latex_metrics_table = create_model_metrics_table(variables_spec, results)
print("""
\\begin{table}[ht]
    \\centering
    \\caption{Modellgüte für alle Variablen}
    \\begin{adjustbox}{max width=\\textwidth}
""")
print(latex_metrics_table)
print("""
    \\end{adjustbox}
\\end{table}

\\vspace{1cm}
""")

# Erstelle und print die LaTeX-Tabellen für die Koeffizienten jeder Variable
for var in variables_spec:
    latex_output = create_latex_table_for_variable(var, results)
    print(latex_output)

In [ ]:
# Anzahl der relevanten Variablen und Layout für die Subplots definieren
n_vars, n_cols, n_rows = len(variables), 2, (len(variables) + 1) // 2

# Subplots erstellen
fig, axes = plt.subplots(n_rows, n_cols, figsize=(10, 4 * n_rows))
axes = axes.flatten()

# Für jede relevante Variable einen Plot erstellen
for i, var in enumerate(variables):
    sns.boxplot(ax=axes[i], x="treatment", y=var, data=grouped_df, palette=treatment_color_dict, hue="treatment", legend=False, dodge=False)
    sns.stripplot(ax=axes[i], x="treatment", y=var, data=grouped_df, dodge=False, c="black", jitter=True, size=5)
    axes[i].set_title(f"Einfluss von Treatment auf {var}")
    axes[i].set_ylabel(var)

# Layout anpassen, Plot anzeigen und speichern
plt.tight_layout()
plt.show()
PLOT_MANAGER.save_plot(fig, filename="effect_of_treatment_with_rope_release", subdir="combined")


Für die betroffenen Parameter wird der Einfluss von `rope_release` mithilfe der bereits angepassten Modelle (`Parameter ~ C(treatment) + rope_release`) entfernt. Dazu werden Vorhersagen für einen konstanten `rope_release`-Wert (den Mittelwert) berechnet und mit den tatsächlichen Werten verglichen. Die daraus resultierenden bereinigten Werte sind frei von Variation, die auf `rope_release` zurückzuführen wäre.

In [ ]:
grouped_adjusted_df = grouped_df.copy()

# Bereinigung: Wir setzen rope_release auf seinen Mittelwert
rope_mean = grouped_df["rope_release"].mean()

for var in relevant_vars_rope_release:
    # Zugehöriges Modellobjekt abrufen
    model = results[var]["model"]

    # Vorhersage mit tatsächlichen rope_release-Werten
    predicted_current = model.predict(grouped_df)

    # Vorhersage, wenn rope_release = rope_mean gesetzt wird
    df_mean_rope = grouped_df.copy()
    df_mean_rope["rope_release"] = rope_mean
    predicted_mean = model.predict(df_mean_rope)

    # Angepasste Werte berechnen:
    actual = grouped_df[var].values
    adjusted = actual + (predicted_mean - predicted_current)
    grouped_adjusted_df[var] = adjusted

In [ ]:
grouped_df.describe()

In [ ]:
grouped_adjusted_df.describe()

##### Latex Tabelle Output


In [ ]:
data_dict

In [ ]:
# DataFrame mit den relevanten Spalten erstellen
variables_latex = [
    'm_amplitude', 
    'm_amplitude_2',
    'initial_amplitude',
    'damping_coeff', 
    'damping_ratio', 
    'frequency_damped', 
    'frequency_undamped',
    'pearson_r',
    'nmae'
]
df_latex = grouped_adjusted_df[variables_latex + ['treatment']].copy()
df_latex.columns

In [ ]:


# Spaltennamen entsprechend der LaTeX-Notation umbenennen
df_latex.rename(columns=data_dict, inplace=True)

# Erstellung deskriptiver Statistiken für alle Beobachtungen
overall_stats = df_latex.describe().drop(index=['count', '25%', '75%'])
overall_stats.rename(index={'50%': 'median'}, inplace=True)

# Erstellung der deskriptiven Statistiken für jede Gruppe
grouped_stats = {
    'overall': overall_stats
}

for treatment, group in df_latex.groupby('treatment', observed=True):
    group_stats = group.describe().drop(index=['count', '25%', '75%'])
    group_stats.rename(index={'50%': 'median'}, inplace=True)
    grouped_stats[treatment] = group_stats

# Zusammenführen der Statistiken in einer Tabelle
combined_stats = pd.concat(grouped_stats, names=['Treatment'])

# LaTeX-Export des kombinierten DataFrames
df_latex_string = combined_stats.to_latex(
    escape=False,
    multirow=True,
    multicolumn=True,
    column_format="l|lrrrrrrrrr", 
    float_format="{:0.2f}".format
)

# LaTeX-Tabellencode erstellen
latex_table = f"""
\\begin{{table}}[h]
    \\centering
    \\caption{{Feldversuch 2 - Ergebnisse, Schwingungsparameter deskriptive Statistiken (Gesamt und gruppiert über Treatment), Amplituden korrigiert über \\texttt{{rope\\_release}}, 9 Beobachtung je Gruppe, jeweils Mittelwert für 4 Elastometer}}
    \\begin{{adjustbox}}{{max width=\\textwidth}}
    {df_latex_string}
    \\end{{adjustbox}}
    \\label{{tab:Feldversuch_2_Deskriptive_Statistiken_Schwingungsparameter}}
\\end{{table}}
"""

print(latex_table)

#### Visualisierung der bereinigten Ergebnisse

Abschließend werden die bereinigten Werte grafisch dargestellt, um die Unterschiede zwischen den Behandlungen in Abwesenheit des `rope_release`-Einflusses zu verdeutlichen. Dies zeigt, wie sich die Treatments auf die Parameter auswirken würden, wenn für alle Einheiten die gleiche mittlere Vorspannung gelten würde.

In [ ]:
relevant_vars_rope_release = ['m_amplitude', 'm_amplitude_2', 'max_strain', 'max_compression']

n_vars = len(relevant_vars_rope_release)
n_cols = 2  # Links Original, rechts angepasst
n_rows = n_vars  # Eine Zeile pro Variable

fig, axes = plt.subplots(n_rows, n_cols, figsize=(10, 4 * n_rows))

for i, var in enumerate(relevant_vars_rope_release):
    # Linke Spalte: Original (grouped_df)
    sns.boxplot(ax=axes[i,0], x="treatment", y=var, data=grouped_df, 
                palette=treatment_color_dict, hue="treatment", legend=False, dodge=False)
    sns.stripplot(ax=axes[i,0], x="treatment", y=var, data=grouped_df, 
                  dodge=False, c="black", jitter=True, size=5)
    axes[i,0].set_title(f"original: {var}")
    axes[i,0].set_ylabel(var)

    # Rechte Spalte: Angepasst (grouped_adjusted_df)
    sns.boxplot(ax=axes[i,1], x="treatment", y=var, data=grouped_adjusted_df, 
                palette=treatment_color_dict, hue="treatment", legend=False, dodge=False)
    sns.stripplot(ax=axes[i,1], x="treatment", y=var, data=grouped_adjusted_df, 
                  dodge=False, c="black", jitter=True, size=5)
    axes[i,1].set_title(f"adjusted : {var}")
    axes[i,1].set_ylabel(var)
    
    # Y-Limits von links holen
    y_min, y_max = axes[i,0].get_ylim()
    # Y-Limits auf rechts anwenden
    axes[i,1].set_ylim(y_min, y_max)

plt.tight_layout()
plt.show()

PLOT_MANAGER.save_plot(fig, filename="effect_of_treatment_comparison", subdir="combined")


Schritt 2: Durchführung von Post-hoc-Tests

Festzustellen welche paarweisen Unterschiede zwischen den Treatments signifikant sind.

In [ ]:
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Wir gehen davon aus, dass grouped_adjusted_df existiert und die Spalten 'treatment' sowie die Variablen aus 'variables' enthält.

for var in variables_spec:
    # Tukey HSD Test durchführen
    # Annahme: Die Spalte 'treatment' enthält die Gruppennamen z.B. 'free', 'gefa_dynamic', 'cobra_static'
    # pairwise_tukeyhsd benötigt die abhängige Variable und die Gruppen.
    tukey_results = pairwise_tukeyhsd(endog=grouped_adjusted_df[var],
                                      groups=grouped_adjusted_df['treatment'],
                                      alpha=0.05)
    
    print(f"--- Post-Hoc Test (Tukey HSD) für Variable: {var} ---")
    print(tukey_results.summary())
    print("\n")


In [ ]:
# Anzahl der relevanten Variablen und Layout für die Subplots definieren
n_vars, n_cols, n_rows = len(variables_spec), 2, (len(variables_spec) + 1) // 2

# Subplots erstellen
fig, axes = plt.subplots(n_rows, n_cols, figsize=(10, 4 * n_rows))
axes = axes.flatten()

# Für jede relevante Variable einen Plot erstellen
for i, var in enumerate(variables_spec):
    sns.boxplot(ax=axes[i], x="treatment", y=var, data=grouped_adjusted_df, palette=treatment_color_dict, hue="treatment", legend=False, dodge=False)
    sns.stripplot(ax=axes[i], x="treatment", y=var, data=grouped_adjusted_df, dodge=False, c="black", jitter=True, size=5)
    axes[i].set_title(f"Einfluss von Treatment auf {var}")
    axes[i].set_ylabel(var)

# Layout anpassen, Plot anzeigen und speichern
plt.tight_layout()
plt.show()
PLOT_MANAGER.save_plot(fig, filename="effect_of_treatment_without_rope_release", subdir="combined")

In [ ]:
# Anzahl der relevanten Variablen und Layout für die Subplots definieren
n_vars, n_cols, n_rows = len(variables_spec), 2, (len(variables_spec) + 1) // 2

# Subplots erstellen
fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, 5 * n_rows))
axes = axes.flatten()

# Für jede relevante Variable einen Plot erstellen
for i, var in enumerate(variables_spec):
    sns.boxplot(ax=axes[i], x="treatment", y=var, data=grouped_adjusted_df, palette=treatment_color_dict, hue="treatment", legend=False, dodge=False)
    sns.stripplot(ax=axes[i], x="treatment", y=var, data=grouped_adjusted_df, dodge=False, c="black", jitter=True, size=5)
    axes[i].set_title(f"Einfluss von Treatment auf {var}")
    axes[i].set_ylabel(var)

# Layout anpassen, Plot anzeigen und speichern
plt.tight_layout()
plt.show()
PLOT_MANAGER.save_plot(fig, filename="effect_of_treatment_without_rope_release_spec", subdir="combined")

#### Vorhergesagte Werte extrahieren und Boxplots für die Sensoren erstellen

In [ ]:
# Vorhergesagte Werte aus den Modellen extrahieren
for variable in variables:
    df[f'predicted_{variable}'] = models[variable].fittedvalues

# Boxplots erstellen mit den vorhergesagten Werten
fig, axes = plt.subplots(4, 2, figsize=(8, 8))
axes = axes.flatten()

for i, variable in enumerate(variables):
    sns.boxplot(ax=axes[i], x='treatment', y=f'predicted_{variable}', data=df, palette=treatment_color_dict, hue='treatment', dodge=False, legend=False)
    axes[i].set_title(f'{variable} by Treatment')
    axes[i].set_xlabel('Treatment')
    axes[i].set_ylabel(f'Predicted {variable}')
plt.tight_layout()
plt.show()
PLOT_MANAGER.save_plot(fig, filename="predicted_effect_for_treatment", subdir="combined")

In [ ]:
def annotate_tukey(ax, tukey_result, significance_level=0.05):
    """
    Fügt eine Textbox mit den Tukey-Test-Ergebnissen und dem festgelegten Signifikanzniveau in den Plot ein.
    
    Parameters:
    ax (matplotlib.axes): Die Achse, auf der der Plot gezeichnet wird.
    tukey_result (TukeyHSDResults): Die Ergebnisse des Tukey HSD Tests.
    significance_level (float): Das Signifikanzniveau, standardmäßig 0.05.
    """
    # Definiere die gewünschte Reihenfolge der Vergleiche
    comparisons_order = [('free', 'gefa_dynamic'), ('free', 'cobra_static'), ('gefa_dynamic', 'cobra_static')]

    # Text für die Annotation zusammenstellen
    text_str = f"Tukey HSD Results: \n(Significance level = {significance_level:.2f})\n\n"
    
    # Durchlaufe die gewünschte Vergleichsreihenfolge
    for group1, group2 in comparisons_order:
        # Filtere die korrekte Paarung aus den Tukey-Ergebnissen
        for i in range(len(tukey_result._results_table.data[1:])):
            pair = tukey_result._results_table.data[i + 1]
            if (pair[0] == group1 and pair[1] == group2) or (pair[0] == group2 and pair[1] == group1):
                p_value = tukey_result.pvalues[i]
                significance = "*" if p_value < significance_level else "n.s."
                text_str += f"\n{group1} vs {group2}: \np = {p_value:.4f} ({significance})\n\n"
    
    # Textbox am Rand des Plots hinzufügen
    ax.annotate(text_str, xy=(1.01, 0.1), xycoords='axes fraction', va='center', ha='left')

# Einzelne Plots für jede Variable erstellen und speichern
for variable in variables:
    fig, ax = plt.subplots(figsize=(8, 4))
    
    # Boxplot für die aktuelle Variable
    sns.boxplot(ax=ax, x='treatment', y=f'predicted_{variable}', data=df, 
                palette=treatment_color_dict, hue='treatment', dodge=False, legend=False)
    
    # Tukey-Test für die aktuelle Variable
    tukey_result = tukey_results[variable]
    
    # Tukey-Ergebnisse annotieren
    annotate_tukey(ax, tukey_result)
    
    ax.set_title(f'{variable} by Treatment')
    ax.set_xlabel('Treatment')
    ax.set_ylabel(f'Predicted {variable}')
    
    plt.tight_layout()
    plt.show()

    # Plot speichern
    plot_filename = f"{variable}_effect_for_treatment"
    PLOT_MANAGER.save_plot(fig, filename=plot_filename, subdir="osc_variables_box")


In [ ]:
# Gruppieren des DataFrames nach 'treatment' und Entfernen unnötiger Spalten
df_treatment_describe = (df.drop(['id', 'ptq_sensor_name'], axis=1)
                         .groupby('treatment', observed=True)
                         .describe())

df_treatment_describe = df_treatment_describe.reset_index()
df_treatment_describe.round(2)

In [ ]:
df.dtypes

In [ ]:
# Gruppieren des DataFrames nach 'treatment' und Entfernen unnötiger Spalten
df_sensor = (df.drop(['id', 'release_force_target', 'ls3_rope_release', 'ls3_cable_max', 'location', 'height', 'diameter', 'direction'], axis=1).
             groupby(['treatment', 'ptq_sensor_name'], observed=True).
             mean())  #.T
#df_sensor.round(4)

In [ ]:
# Gruppieren des DataFrames und Anwenden von mean für ptq_sensor_name 
df_id = ((df.drop(['ptq_sensor_name', 'location', 'height', 'diameter', 'direction'], axis=1)
          .groupby(['treatment', 'id'], observed=True)
          .mean())
         .reset_index())

df_id.round(4).head()

### Zusammenhangsanalyse für LS3 und PTQ

In [ ]:
# Auswahl der neuen Spaltennamen für die Korrelationsmatrix
columns_corr = ['ptq_m_amplitude',
                'ptq_m_amplitude_2',
                'ptq_initial_amplitude',
                'ptq_damping_coeff',
                'ptq_angular_frequency',
                'ptq_y_shift',
                'ptq_pearson_r',
                #'ptq_nrmse',
                'ptq_nmae',
                'release_force_target',
                'ls3_rope_release',
                'ls3_cable_max']
df_corr = df_id.copy()[columns_corr]

# Berechnung der Korrelationsmatrix
correlation_matrix = df_corr.corr()

# Visualisierung der Korrelationsmatrix mit Seaborn
fig1, ax = plt.subplots(figsize=(8, 8))  # Anpassen der Größe der Grafik
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', ax=ax, annot_kws={'size': 10})

# Titel und Schriftgrößen anpassen
#plt.title('Correlation Matrix for LS3 and PTQ', fontsize=18)
ax.set_xticklabels(ax.get_xmajorticklabels(), fontsize=12, rotation=45, ha='right')
ax.set_yticklabels(ax.get_ymajorticklabels(), fontsize=12, rotation=45, ha='right')
ax.set_xlabel('')
ax.set_ylabel('')
plt.tight_layout()
plt.show()
PLOT_MANAGER.save_plot(fig1, filename="correlation_matrix", subdir="combined")

In [ ]:
# ANOVA für 'ls3_rope_release'
model_rope_release = smf.ols('ls3_rope_release ~ treatment', data=df_id).fit()
anova_rope_release = sm.stats.anova_lm(model_rope_release, typ=2)

# ANOVA für 'ls3_cable_max'
model_cable_max = smf.ols('ls3_cable_max ~ treatment', data=df_id).fit()
anova_cable_max = sm.stats.anova_lm(model_cable_max, typ=2)

# Zusammenfassungen der Modelle
summary_rope_release = model_rope_release.summary()
summary_cable_max = model_cable_max.summary()
# Zusammenfassungen der Modelle
summary_rope_release_latex = model_rope_release.summary().as_latex()
summary_cable_max_latex = model_cable_max.summary().as_latex()

#print(summary_rope_release_latex)
#print(summary_cable_max_latex)

anova_rope_release, summary_rope_release, anova_cable_max, summary_cable_max

In [ ]:
# Funktion zur Erstellung des Text-Strings für die statistischen Parameter
def annotate_stats(x, y):
    slope, intercept, r_value, p_value, std_err = linregress(x, y)
    text_str = f"R = {r_value:.2f}\nSlope = {slope:.2f}\nIntercept = {intercept:.2f}\np-value = {p_value:.2e}\nStd Err = {std_err:.2f}"
    return text_str

#### Zusammenhang Ziel- und Ist-Vorspannung

In [ ]:
fig2, ax1 = plt.subplots(figsize=(8, 5))
sns.regplot(x='release_force_target', y='ls3_rope_release', data=df_id, ax=ax1, color='b', ci=95)
ax2 = ax1.twinx()
ax2.axis('off')
stats_text = annotate_stats(df_id['release_force_target'], df_id['ls3_rope_release'])
ax2.annotate(stats_text, xy=(1.01, 0.1), xycoords='axes fraction')
#ax1.set_title('Correlation Between Target and Actual Release Forces"')
ax1.set_xlabel('Release Force Target [kN]')
ax1.set_ylabel('Release Force [kN]')
plt.tight_layout()
plt.show()
PLOT_MANAGER.save_plot(fig2, filename=f"release_force_target_vs_ls3_rope_release", subdir="combined")

#### Zusammenhang Vorspannung und resultierende Lastspitzen

In [ ]:
fig3, ax1 = plt.subplots(figsize=(8, 5))
ax2 = ax1.twinx()
ax2.axis('off')
y_pos_init = 0.1
for idx, (treatment, color) in enumerate(treatment_color_dict.items()):
    subset = df_id[df_id['treatment'] == treatment]
    if subset['ls3_cable_max'].isna().all():
        continue
    sns.regplot(x='ls3_rope_release', y='ls3_cable_max', data=subset, ax=ax1, color=color, label=treatment, ci=95)
    stats_text = annotate_stats(subset['ls3_rope_release'], subset['ls3_cable_max'])
    ax2.annotate(f"{treatment}:\n{stats_text}", xy=(1.01, y_pos_init + idx * 0.3), xycoords='axes fraction')
#ax1.set_title('Correlation Between Release Force and Peak Cable Force')
ax1.set_xlabel('Release Force [kN]')
ax1.set_ylabel('Peak Force in Cable [kN]')
ax1.legend(title='Treatment', loc='upper left')
plt.tight_layout()
plt.show()
PLOT_MANAGER.save_plot(fig3, filename=f"release_force_vs_ls3_cable_max", subdir="combined")

In [ ]:
fig3, ax1 = plt.subplots(figsize=(8, 5))
ax2 = ax1.twinx()
ax2.axis('off')
y_pos_init = 0.1
for idx, (treatment, color) in enumerate(treatment_color_dict.items()):
    subset = df_id[df_id['treatment'] == treatment]
    if subset['ptq_m_amplitude'].isna().all():
        continue
    sns.regplot(x='ls3_rope_release', y='ptq_m_amplitude', data=subset, ax=ax1, color=color, label=treatment,
                ci=95)
    stats_text = annotate_stats(subset['ls3_rope_release'], subset['ptq_m_amplitude'])
    ax2.annotate(f"{treatment}:\n{stats_text}", xy=(1.01, y_pos_init + idx * 0.3), xycoords='axes fraction')
#ax1.set_title('Correlation Between Release Force and Elongation Amplitude')
ax1.set_xlabel('Release Force [kN]')
ax1.set_ylabel('Elongation Amplitude [$\mu$m] (mean for all Sensors)')
ax1.legend(title='Treatment', loc='upper left')
plt.tight_layout()
plt.show()
PLOT_MANAGER.save_plot(fig3, filename=f"ls3_release_force_vs_ptq_m_amplitude", subdir="combined")


In [ ]:
fig3, ax1 = plt.subplots(figsize=(8, 5))
ax2 = ax1.twinx()
ax2.axis('off')
y_pos_init = 0.1
for idx, (treatment, color) in enumerate(treatment_color_dict.items()):
    subset = df_id[df_id['treatment'] == treatment]
    if subset['ptq_m_amplitude_2'].isna().all():
        continue
    sns.regplot(x='ls3_rope_release', y='ptq_m_amplitude_2', data=subset, ax=ax1, color=color, label=treatment,
                ci=95)
    stats_text = annotate_stats(subset['ls3_rope_release'], subset['ptq_m_amplitude_2'])
    ax2.annotate(f"{treatment}:\n{stats_text}", xy=(1.01, y_pos_init + idx * 0.3), xycoords='axes fraction')
#ax1.set_title('Correlation Between Release Force and Elongation Amplitude 2')
ax1.set_xlabel('Release Force [kN]')
ax1.set_ylabel('Elongation Amplitude 2 [$\mu$m] (mean for all Sensors)')
ax1.legend(title='Treatment', loc='upper left')
plt.tight_layout()
plt.show()
PLOT_MANAGER.save_plot(fig3, filename=f"ls3_release_force_vs_ptq_m_amplitude_2", subdir="combined")

In [ ]:
# Funktion zur Durchführung des ANOVA-Tests und Berechnung der Effektstärke (Eta Squared)
def perform_anova_and_effect_size(df: pd.DataFrame, variable: str, treatments: List[str]) -> str:
    groups = [df[df['treatment'] == treatment][variable].dropna() for treatment in treatments]
    f_stat, p_value = f_oneway(*groups)

    # Berechnung der Effektstärke (Eta Squared)
    n = sum([len(g) for g in groups])
    ss_total = sum([(x - df[variable].mean()) ** 2 for g in groups for x in g])
    eta_squared = f_stat * len(groups) / (f_stat * len(groups) + (n - len(groups)))

    # Überprüfung der Signifikanz
    significance = "*" if p_value < 0.05 else ""

    return f"{variable}: {significance}\nF-statistic = {f_stat:.2f}\np-value = {p_value:.2e}\nEta Squared = {eta_squared:.2f}"

In [ ]:
# Funktion zur Erstellung von Boxplots
def create_boxplot(df: pd.DataFrame, variable: str, group_by: str, ax: plt.Axes, color_dict: Dict[str, str], perform_stats: bool) -> None:
    valid_df = df.dropna(subset=[variable])
    sns.boxplot(x=group_by, y=variable, hue=group_by, data=valid_df, ax=ax, palette=color_dict, dodge=False)
    ax2 = ax.twinx()
    ax2.axis('off')
    if perform_stats:
        stats_text = perform_anova_and_effect_size(valid_df, variable, valid_df[group_by].unique())
        ax2.annotate(stats_text, xy=(1.01, 0.1), xycoords='axes fraction')
    ax.set_title(f'Einfluss von {group_by} auf {variable}')
    ax.set_xlabel(group_by)
    ax.set_ylabel(variable)

# Funktion zur Erstellung kombinierter Plots
def create_combined_plot(df: pd.DataFrame, columns: List[str], group_by: str, color_dict: Dict[str, str], num_columns: int = 3, perform_stats: bool = False) -> None:
    num_rows = len(columns) // num_columns + (len(columns) % num_columns > 0)
    fig, axes = plt.subplots(num_rows, num_columns, figsize=(16, 4 * num_rows))
    axes = axes.flatten()

    for idx, variable in enumerate(columns):
        create_boxplot(df, variable, group_by, axes[idx], color_dict, perform_stats)

    plt.tight_layout()
    plt.show()
    PLOT_MANAGER.save_plot(fig, filename=f"combined_plot_{group_by}", subdir="combined")

# Funktion zur Erstellung einzelner Plots
def create_individual_plots(df: pd.DataFrame, columns: List[str], group_by: str, color_dict: Dict[str, str], perform_stats: bool = False) -> None:
    for variable in columns:
        fig, ax = plt.subplots(figsize=(8, 5))
        create_boxplot(df, variable, group_by, ax, color_dict, perform_stats)
        plt.tight_layout()
        #plt.show()
        PLOT_MANAGER.save_plot(fig, filename=f"{group_by}_{variable}", subdir="individual_plots")

In [ ]:
columns = ['ptq_m_amplitude',
           'ptq_m_amplitude_2',
           'ptq_initial_amplitude',
           'ptq_damping_coeff',
           'ptq_angular_frequency',
           'ptq_y_shift',
           'ptq_pearson_r',
           #'ptq_nrmse',
           #'ptq_nmae',
           #'release_force_target',
           'ls3_rope_release',
           'ls3_cable_max'
           ]

# Beispiel: Erstellen von Plots gruppiert nach 'treatment'
create_combined_plot(df, columns, 'treatment', treatment_color_dict, perform_stats=True)
create_individual_plots(df, columns, 'treatment', treatment_color_dict, perform_stats=True)

In [ ]:
# Beispiel: Erstellen von Plots gruppiert nach 'ptq_sensor_name'
columns = ['ptq_m_amplitude', 'ptq_m_amplitude_2', 'ptq_initial_amplitude', 'ptq_damping_coeff', 'ptq_angular_frequency', 'ptq_pearson_r']

create_combined_plot(df, columns, 'ptq_sensor_name', sensor_color_dict)
create_individual_plots(df, columns, 'ptq_sensor_name', sensor_color_dict)